In [1]:
import boto3
from botocore.exceptions import ClientError
import json
import os
import datetime
import pytz
import calendar 
import decimal

In [2]:
# 先設定epurl變數為sqs container連線位置，此變數後續可直接在endpoint_url使用
epurl1="http://cc104.sns.local:9911"
epurl2="http://cc104.sqs.local:9324"
#============================================
# 先取得local time,utc time
tz = pytz.timezone('Asia/Taipei')
utc = pytz.utc
i=1
#============================================
#取得local time,utc time
#print(pytz.country_timezones('tw'))
tz = pytz.timezone('Asia/Taipei')
utc = pytz.utc
#============================================

In [3]:
#============================================
#取得local time,utc time
#print(pytz.country_timezones('tw'))
tz = pytz.timezone('Asia/Taipei')
utc = pytz.utc
print("="*40)
# 用boto3的session方法調用container中.aws/目錄下的環境變數設定檔，並存入變數user_session中
user_session=boto3.session.Session(profile_name='user1')
#============================================
# boto3透過container的3個aws環境變數，來連線sqs
sns_client = user_session.client('sns', endpoint_url=epurl1)
print("Local time:",datetime.datetime.now(tz))
print("  UTC time: ",datetime.datetime.now(pytz.utc),"\n",sns_client,"~~連線成功~~")
print("="*40)
# ============================================
# boto3透過container的3個aws環境變數，來連線sqs
sqs_client = user_session.client('sqs', endpoint_url=epurl2)
print("Local time:",datetime.datetime.now(tz))
print("  UTC time: ",datetime.datetime.now(pytz.utc),"\n",sqs_client,"~~連線成功~~")
print("="*40)
# ============================================
sqs_reso = user_session.resource('sqs', endpoint_url=epurl2)

Local time: 2019-02-10 17:07:36.796133+08:00
  UTC time:  2019-02-10 09:07:36.796595+00:00 
 <botocore.client.SNS object at 0x7f08b6c09048> ~~連線成功~~
Local time: 2019-02-10 17:07:36.814608+08:00
  UTC time:  2019-02-10 09:07:36.816105+00:00 
 <botocore.client.SQS object at 0x7f08b6b824a8> ~~連線成功~~


In [5]:
#定義sqs隊列名稱
QName='test17'
# 用create_queue方法創建test17隊列
try:
    queue = sqs_client.create_queue(QueueName=QName, Attributes={'DelaySeconds': '5'})
    #輸出回應內容
    print("="*40)
    print("已創建sqs隊列(QueueName為",QName,")\nQueueUrl： ",queue['QueueUrl'],"\nHTTPStatusCode：",queue['ResponseMetadata']['HTTPStatusCode'])
    print("="*40)
except:
    print(".....")

.....


In [7]:
#創建SNS主題
Name1='my-topic1'
Name2='my-topic2'
Name3='my-topic3'
topic1 ={'1': 'hahaha'}
topic2 ={'2': 'hehehe'}
topic3 ={'3': 'wahaha'}
response1 = sns_client.create_topic(Name=Name1,Attributes=topic1)
response2 = sns_client.create_topic(Name=Name2,Attributes=topic2)
response3 = sns_client.create_topic(Name=Name3,Attributes=topic3)
print("="*40)
print("已建立主題:")
print("  ",Name1,"\n\t  topicArn:\n\t\t\t",response1['TopicArn'])
print("  ",Name2,"\n\t  topicArn:\n\t\t\t",response2['TopicArn'])
print("  ",Name3,"\n\t  topicArn:\n\t\t\t",response3['TopicArn'])
print("="*40)

已建立主題:
   my-topic1 
	  topicArn:
			 arn:aws:sns:us-east-1:123456789012:my-topic1
   my-topic2 
	  topicArn:
			 arn:aws:sns:us-east-1:123456789012:my-topic2
   my-topic3 
	  topicArn:
			 arn:aws:sns:us-east-1:123456789012:my-topic3


In [8]:
ptcl='sqs'
#============================================
#將sqs的test17隊列訂閱主題my-topic1
response = sns_client.subscribe(
    TopicArn=response1['TopicArn'],
    Protocol=ptcl,
    Endpoint='aws-sqs://'+QName+'?amazonSQSEndpoint=http://cc104.sqs.local:9324&accessKey=&secretKey=',
    Attributes={
        'action': 'hahahahahah'
    },
    ReturnSubscriptionArn=True
)
print(ptcl,QName,"隊列，已訂閱",response1['TopicArn'],"的主題:")
print("="*40)
print("  ARN:\n\t",response['SubscriptionArn'])
print("="*40)
print("  Metadata:\n\t",response['ResponseMetadata'])
print("="*40)

sqs test17 隊列，已訂閱 arn:aws:sns:us-east-1:123456789012:my-topic1 的主題:
  ARN:
	 arn:aws:sns:us-east-1:123456789012:my-topic1:397bfc7d-1514-44ca-b54b-ced57b3136ed
  Metadata:
	 {'RequestId': '643c0ffa-f585-4962-b50d-af19cc413894', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'akka-http/10.0.10', 'date': 'Sun, 10 Feb 2019 09:08:56 GMT', 'content-type': 'text/xml; charset=UTF-8', 'content-length': '335'}, 'RetryAttempts': 0}


In [9]:
sqsarn=response1['TopicArn']
#============================================
#推送訊息至test17隊列
resp = sns_client.publish(
    TopicArn=sqsarn,
    Subject ='sns_test',
    Message='Hello World~~~Wahahaha',
    MessageAttributes={
        'event_type': {
            'DataType': 'string',
            'StringValue': 'product'
        }
    }
)
#輸出訊息MessageId、ResponseMetadata
print("="*40)
print("推送訊息MessageId:\n\t",resp['MessageId'])
print("="*40)
print("ResponseMetadata:\n\t",resp['ResponseMetadata'])
print("="*40)

推送訊息MessageId:
	 90c119c9-365a-4019-9e6e-3c1f47f327e6
ResponseMetadata:
	 {'RequestId': '7d57b27a-e9c3-4c60-b601-fe9f84196e0b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'akka-http/10.0.10', 'date': 'Sun, 10 Feb 2019 09:09:00 GMT', 'content-type': 'text/xml; charset=UTF-8', 'content-length': '270'}, 'RetryAttempts': 0}


In [10]:
queue1 = sqs_reso.get_queue_by_name(QueueName='test17')

#將test隊列中的message取出(一次只能rolling出一個`,有時會'無')
for d in queue1.receive_messages():
    try:
        print('隊列:\t',d.queue_url)
        print("="*40)
        print('訊息 Body內容:\t',d.body)
        print("="*40)
        ma = json.loads(d.body)
        print('訊息 Message內容:\t',ma['Message'])
        print("="*40)
        print('訊息 ID:\t',d.message_id)
        print('訊息 md5 Body:\t',d.md5_of_body)
        print("="*40)
        d.delete()
    except:
        print(".....")

隊列:	 http://cc104.sqs.local:9324/queue/test17
訊息 Body內容:	 {"MessageId":"90c119c9-365a-4019-9e6e-3c1f47f327e6","Message":"Hello World~~~Wahahaha","Type":"Notification"}
訊息 Message內容:	 Hello World~~~Wahahaha
訊息 ID:	 ebc045eb-0e0a-4c2e-8112-51c6f62b2de7
訊息 md5 Body:	 8aa5fa2423a469ccfebb86119ecb5663


In [11]:
i=0
#已清單的方式列印出訂閱主題
lsubscript=sns_client.list_subscriptions().get('Subscriptions')
for jj in lsubscript:
    try:
        print("訂閱主題",i+1,': ',jj['SubscriptionArn'].split(":")[5],'\n\tArn:',jj['SubscriptionArn'],'\n\tProtocol:',jj['Protocol'],'\n\tEndpoint:',jj['Endpoint'],"\n\n")
        i+=1
    except:
        print(".....")

訂閱主題 1 :  my-topic1 
	Arn: arn:aws:sns:us-east-1:123456789012:my-topic1:397bfc7d-1514-44ca-b54b-ced57b3136ed 
	Protocol: sqs 
	Endpoint: aws-sqs://test17?amazonSQSEndpoint=http://cc104.sqs.local:9324&accessKey=&secretKey= 


訂閱主題 2 :  my-topic1 
	Arn: arn:aws:sns:us-east-1:123456789012:my-topic1:25ece3e4-02f2-4d5f-b7e9-95faac806bf7 
	Protocol: sqs 
	Endpoint: aws-sqs://test17?amazonSQSEndpoint=http://cc104.sqs.local:9324&accessKey=&secretKey= 


訂閱主題 3 :  my-topic2 
	Arn: arn:aws:sns:us-east-1:123456789012:my-topic2:c921e1c1-5308-4388-a74b-6896dc495f28 
	Protocol: sqs 
	Endpoint: aws-sqs://test17?amazonSQSEndpoint=http://cc104.sqs.local:9324&accessKey=&secretKey= 


訂閱主題 4 :  my-topic3 
	Arn: arn:aws:sns:us-east-1:123456789012:my-topic3:c7c1766b-ce90-4853-a8f9-abac466e3b64 
	Protocol: sqs 
	Endpoint: aws-sqs://test17?amazonSQSEndpoint=http://cc104.sqs.local:9324&accessKey=&secretKey= 


